In [1]:
import pandas as pd

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [3]:
ratings_group = ratings.groupby('userId').agg({'rating':'count', 'timestamp':{'min','max'}})
ratings_group['lifetime'] = ratings_group.timestamp['max'] - ratings_group.timestamp['min']
ratings_group = ratings_group[ratings_group.rating['count'] > 100][['rating', 'lifetime']]
lifetime_avg = pd.to_timedelta(ratings_group['lifetime'].mean(),  unit='s')
lifetime_avg

Timedelta('463 days 21:28:27.449612')

In [4]:
print(f'Среднее время жизни пользователей, выставивших более 100 оценок, \
{lifetime_avg.days} дня и {lifetime_avg.components.hours} час')

Среднее время жизни пользователей, выставивших более 100 оценок, 463 дня и 21 час


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [8]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [9]:
# Таблица с тремя типами выручки для каждого client_id без указания адреса клиента
client_revenue = client_base[['client_id']].merge(rzd, how='outer').merge(auto, how='outer').merge(air, how='outer')
client_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [10]:
# Таблица с тремя типами выручки для каждого client_id c указанием адреса клиента
client_revenue = client_base.merge(rzd, how='left').merge(auto, how='left').merge(air, how='left')
client_revenue

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [11]:
# Проверка
print(
    len(client_revenue) == len(client_base),
    client_revenue.rzd_revenue.sum() == rzd.rzd_revenue.sum(),
    client_revenue.auto_revenue.sum() == auto.auto_revenue.sum(),
    client_revenue.air_revenue.sum() == air.air_revenue.sum()
)

True True True True


### Задание 3

   В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

----------------------------

Предже всего в таблице визитов и покупок должны быть указаны даты визитов и покупок. 
Также в последовательности координат должны быть указаны даты на которые определены эти координаты.
В датах должен быть индентификатов часового пояса, либо они все должны быть в формате UTC.

Затем таблицу визитов и покупок с координатами нужно связывать по id пользователя и по дате.
Возможно, даты определения координат не будут совпадать с датами визитов и покупок, тогда нам нужно будет определеять координаты по ближайшей дате получения координат пользователя.

Из этих координат можно получить местоположение пользователей, определить где находятся наши клиенты, 
какие у них потребности, их платежеспособность в зависимости от их локации. 
А это нужно будет для оптимизации логистики, рекламы, структуры подразделений (например, оптимизации местоположения пунктов выдачи товара), товара (возможно, наши пользователи часто меняют свои координаты, тогда им понадобятся товары для путешествий)